In [1]:
import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os 
from sklearn.model_selection import cross_val_score
import os
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import StratifiedKFold

In [2]:
input_path = "/kaggle/input/icr-identify-age-related-conditions/"
work_path = os.getcwd()
df_train_file = pd.read_csv(input_path + 'train.csv')
df_greek_file = pd.read_csv(input_path + 'greeks.csv')
df_train_file.sample(10)

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
374,91004fe0a0fd,0.337567,3237.48084,85.200147,27.996880,12.436146,6.697404,0.108706,7.026730,7.418054,...,9.441877,0.57159,0.250601,11.701916,216.936538,11180.175820,22.586470,166.292660,0.256979,0
380,93205d43a401,0.858873,7679.56629,85.200147,50.403675,8.138688,6.130428,0.062727,9.780704,1.229900,...,12.062681,1.54396,2.506010,19.134214,143.488453,11356.130910,25.652574,86.691832,0.048120,0
538,da1a5aec1818,0.465757,3487.90244,85.200147,20.235798,26.712936,7.414983,0.056637,3.396778,1.229900,...,0.173229,0.49706,0.257374,11.999580,72.611063,14737.274460,22.627401,64.896704,21.978000,1
227,5c03d177da66,0.299110,1561.99021,85.200147,12.939266,8.138688,4.358628,0.025578,12.440148,2.892022,...,2.069300,1.80177,0.067730,18.817946,72.611063,20685.982260,38.158855,22.458704,0.371250,0
341,860833bfe713,0.376024,2967.65810,85.200147,34.262111,8.138688,5.643183,0.025578,9.200920,1.229900,...,3.536792,1.65184,0.108368,21.041124,103.865755,4255.175916,36.796969,69.122500,0.143550,0
194,49e14acf8ffb,0.952879,14503.25602,85.200147,200.976718,15.013572,13.447962,0.025578,38.971568,7.534016,...,3.868904,1.60892,0.853398,34.129038,198.464644,2878.004520,24.115801,122.301172,0.078025,0
105,2bb2902bd5e1,0.275608,3799.72912,85.200147,26.363212,8.138688,5.186945,0.025578,9.475057,1.229900,...,7.192990,1.44130,0.507975,16.390124,72.611063,11744.478680,32.997828,42.346694,0.083935,0
114,2e4e80b95e24,0.170920,1963.64850,85.200147,22.574033,8.138688,2.595687,0.029232,3.396778,1.229900,...,0.173229,1.09620,0.067730,25.636312,198.146620,6926.005926,25.287916,73.265992,21.978000,0
445,b002a6adb6be,0.418754,6031.82822,91.801569,58.397032,9.271392,17.877462,0.025578,15.194122,11.989768,...,0.173229,2.30898,0.230282,28.389704,95.548544,13815.685780,25.526060,101.552848,21.978000,0
138,37342e4943f2,0.286291,192.59328,85.200147,6.466536,8.138688,5.014194,0.052374,3.396778,1.229900,...,5.269528,0.49706,0.189644,18.650510,72.611063,8050.058316,22.891592,15.110500,0.213729,0


In [3]:
def prepare_initial(df, df_greek_f):
    df.sort_values(by=['Id'], axis=0, ascending=True, inplace=True)
    
    # training set labels
    df_y = None
    if 'Class' in df.columns:
      df_y = df['Class']
    
    # training set input
    df['EJ_dummy'] = (df['EJ'] == 'B')
    df['EJ_dummy'] = df['EJ_dummy'].astype('float64')
    temp_col = df['EJ_dummy']
    df_input = df.drop(['Id', 'EJ', 'EJ_dummy'], axis=1)
    if 'Class' in df.columns:
        df_input = df_input.drop(['Class'], axis=1)
    for i in df_input.columns:
        df_input[i].fillna(df_input[i].mode()[0], inplace=True)
    
    #scaler = StandardScaler()
    scaler = RobustScaler()
    df_x = pd.DataFrame(scaler.fit_transform(df_input), columns=df_input.columns)
    df_x['EJ_dummy'] = temp_col
    #print(df_x.head())
    
    #print(df_y.head(10))
    
    # additional data
    df_greek_f.sort_values(by=['Id'], axis=0, ascending=True, inplace=True)
    df_greek_f.loc[df_greek_f['Epsilon'] == 'Unknown', 'Epsilon'] = pd.to_datetime('1/1/2021')
    df_greek_f['Epsilon_date'] = pd.to_datetime(df_greek_f.Epsilon)
    df_greek_f['Age'] = pd.to_datetime('1/1/2023') - df_greek_f.Epsilon_date
    df_greek_f['Age_days'] = df_greek_f['Age'].dt.total_seconds() / (24 * 60 * 60)
    
    scaler = RobustScaler()
    df_greek_f[['Age_days']] = pd.DataFrame(scaler.fit_transform(df_greek_f[['Age_days']]), columns=['Age_days'])
    
    df_greek_x = pd.get_dummies(df_greek_f, columns = ['Alpha', 'Beta', 'Gamma', 'Delta'])
    df_greek_x = df_greek_x.drop(['Age_days', 'Alpha_A', 'Alpha_B', 'Alpha_D', 'Alpha_G', 'Age', 'Epsilon_date', 'Id', 'Epsilon'], axis=1)
    # Age_days must be dropped for RF multy categories
    #df_greek_x = df_greek_x.loc[:, (df_greek_x.columns != 'Alpha_A') & (df_greek_x.columns != 'Age') & (df_greek_x.columns != 'Epsilon_date') & (df_greek_x.columns != 'Id') & (df_greek_x.columns != 'Epsilon')]
    #print(df_greek_x.head())
    
    # imp_mean = SimpleImputer(missing_values=pd.NA, strategy='most_frequent')
    # imp_mean.fit_transform(df_x)
    return df_x, df_y, df_greek_x

In [4]:
df_ready_x, df_ready_y, df_ready_greek = prepare_initial(df_train_file, df_greek_file)

In [5]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=13, random_state=0).fit(df_ready_x, df_ready_y)
scores = cross_val_score(clf, df_ready_x, df_ready_y, cv=StratifiedKFold(n_splits=5))
scores.mean()

0.9157487542617361

In [6]:
forest_importances = pd.Series(clf.feature_importances_, index=df_ready_x.columns)
top_features = list(forest_importances.nlargest(22).index)
top_features

['DU',
 'FL',
 'CR',
 'GL',
 'DI',
 'DA',
 'EH',
 'DE',
 'FR',
 'AB',
 'AF',
 'BC',
 'FE',
 'DH',
 'CC',
 'FD ',
 'AM',
 'EE',
 'CD ',
 'FI',
 'BR',
 'GH']

In [7]:
df_reduced_x = df_ready_x.loc[:, top_features]
clf = RandomForestClassifier(max_depth=13, random_state=0).fit(df_reduced_x, df_ready_y)
df_reduced_x

,DU,FL,CR,GL,DI,DA,EH,DE,FR,AB,...,FE,DH,CC,FD,AM,EE,CD,FI,BR,GH
0,4.822602,0.705590,-2.452321,-0.010025,-0.646331,0.848132,3.688312,-0.037417,0.598571,-0.472222,...,0.306960,-0.562500,-0.452925,1.831606,0.069272,-0.278075,-1.608224,-2.125230,0.356865,-0.716263
1,-0.231275,-0.469346,1.434529,0.990161,-0.308376,0.921214,-0.350649,-0.404150,-0.624571,-0.680556,...,-0.102154,0.041667,-0.828326,-0.343264,0.611687,-0.777184,-0.831902,0.138122,-1.046384,-0.124567
2,0.992116,0.773432,-0.112872,-0.006520,-0.158303,0.922123,1.246753,0.043616,-0.153143,0.375000,...,0.181235,-1.125000,-0.775229,1.500000,0.440180,2.445633,0.174876,0.561694,-0.263822,-0.218622
3,2.293035,0.511662,-0.351126,-0.008401,0.154814,-0.081184,2.259740,-0.347566,-0.624571,-0.333333,...,0.660302,-0.500000,0.282914,1.312176,2.105694,0.529412,0.237686,1.639042,0.061928,0.789556
4,0.854139,0.846567,-0.139561,-0.011111,-0.508930,1.060433,0.337662,-0.336377,46.670286,0.083333,...,1.614526,-1.145833,-0.581666,0.524611,-0.239281,0.386809,-0.204474,1.243094,9.892594,1.248820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,-0.231275,-0.469346,-0.120656,0.990161,0.743298,-1.168568,-0.350649,0.151753,0.128000,-0.666667,...,1.791274,0.666667,0.156026,-0.343264,-0.410097,-0.557932,-0.702666,-0.022099,-0.475613,-0.504561
613,0.381078,1.187935,0.112038,-0.008881,0.990727,-0.224620,0.233766,-0.470464,0.109714,0.263889,...,2.027103,0.604167,0.542277,0.915803,0.968303,-0.823529,0.154315,0.322284,-0.222299,-0.081158
614,-0.231275,-0.469346,0.552405,0.990161,1.407580,2.362517,-0.350649,-0.264198,-0.624571,0.236111,...,-0.411472,0.187500,0.238756,-0.343264,1.295989,-0.172906,1.791460,0.696133,2.644947,-0.375590
615,0.249671,1.028612,-0.547401,-0.007084,-0.261741,0.079282,0.233766,-0.612112,-0.338286,0.027778,...,-0.160651,1.458333,-0.268142,0.943005,0.117335,-0.436720,1.228875,-0.640884,0.172448,-0.464297


In [8]:
df_test_file = pd.read_csv(input_path + 'test.csv')
res_x, res_y, res_greek = prepare_initial(df_test_file, df_greek_file)
res_reduced_x = res_x.loc[:, top_features]
preds = pd.DataFrame(clf.predict_proba(res_reduced_x))
preds

,0,1
0,0.999941,0.000059
1,0.999941,0.000059
2,0.999941,0.000059
3,0.999941,0.000059
4,0.999941,0.000059


In [9]:
df_test_y = pd.concat([df_test_file['Id'], preds], axis=1)
df_test_y

,Id,0,1
0,00eed32682bb,0.999941,0.000059
1,010ebe33f668,0.999941,0.000059
2,02fa521e1838,0.999941,0.000059
3,040e15f562a2,0.999941,0.000059
4,046e85c7cc7f,0.999941,0.000059


In [10]:
df_test_y.to_csv('/kaggle/working/submission.csv', header=['Id', 'class_0', 'class_1'], index=False)
!head /kaggle/working/submission.csv

Id,class_0,class_1
00eed32682bb,0.9999411764705883,5.882352941176471e-05
010ebe33f668,0.9999411764705883,5.882352941176471e-05
02fa521e1838,0.9999411764705883,5.882352941176471e-05
040e15f562a2,0.9999411764705883,5.882352941176471e-05
046e85c7cc7f,0.9999411764705883,5.882352941176471e-05
